# Chapter 5: Querying Multiple Tables

As seen in the previous chapters, joining tables is often or primarily required for information gathering

In [1]:
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine, URL, select, func
from sqlalchemy.orm import Session
import pandas as pd

from utils import print_sql_statement


load_dotenv()

url_object = URL.create(
    os.environ["DB_ENGINE"],
    username=os.environ["DB_USER"],
    password=os.environ["DB_PASSWD"],
    host=os.environ["DB_HOST"],
    database=os.environ["DB_NAME"],
)

engine = create_engine(url_object)

# Joining Three Tables

Find all the accounts opened by 'experienced' (hired prior to 2003) tellers currently assigned to the 'Woburn Branch'

This is a little complicated, but the logic is
    * Get all accounts
    * Join all accounts with all the 'experienced' tellers
    * Join all experienced tellers at the Woburn branch

In [22]:
from datetime import date

from sqlalchemy import and_
from sqlalchemy.orm import aliased

from model import Account, Branch, Employee


with Session(engine) as session:
    df = pd.read_sql_query(
        """
        SELECT
            a.account_id
            , a.cust_id
            , a.open_date
            , a.product_cd
        FROM
            account a
        -- ASSOICATE ALL EXPERIENCED TELLERS WITH ACCOUNTS
        JOIN (
            SELECT
                emp_id
                , assigned_branch_id
            FROM
                employee
            WHERE
                employee.start_date <= '2003-01-01'
                AND
                employee.title LIKE '%Teller%'
        ) e ON e.emp_id = a.open_emp_id
        -- ASSOCIATED ALL EMPLOYEES AT THE WOBURN BRANCH
        JOIN (
            SELECT
                branch_id
            FROM
                branch
            WHERE
                branch.name = 'Woburn Branch'
        ) b ON b.branch_id = e.assigned_branch_id
        ;
        """,
        con=session.connection()
    )

    # Get the employee sub-query
    employee_subquery = (
        select(
            Employee.emp_id,
            Employee.assigned_branch_id
        )
        .select_from(Employee)
        .where(
            and_(
                Employee.title.like("%Teller%"),
                Employee.start_date <= date(2003, 1, 1)
            )
        )
        .subquery("e")
    )
    employee_alias = aliased(Employee, employee_subquery)

    # Get the branch sub-query
    branch_subquery = (
        select(
            Branch.branch_id
        )
        .select_from(Branch)
        .where(Branch.name == "Woburn Branch")
        .subquery("b")
    )
    branch_alias = aliased(Branch, branch_subquery)

    statement = (
        # Get all accounts
        select(
            Account.account_id,
            Account.cust_id,
            Account.open_date,
            Account.product_cd
        )
        .select_from(Account)
        # Associate all accounts with the experienced teller
        .join(
            employee_subquery,
            Account.open_emp_id == employee_alias.emp_id
        )
        # Associated all the tellers at the Woburn Branch
        .join(
            branch_subquery,
            branch_alias.branch_id == employee_alias.assigned_branch_id
        )
    )
    print_sql_statement(statement)
    results = session.execute(statement).all()

print(df)
print(results)

"""SELECT account.account_id, account.cust_id, account.open_date, account.product_cd 
FROM account JOIN (SELECT employee.emp_id AS emp_id, employee.assigned_branch_id AS assigned_branch_id 
FROM employee 
WHERE employee.title LIKE :title_1 AND employee.start_date <= :start_date_1) AS e ON account.open_emp_id = e.emp_id JOIN (SELECT branch.branch_id AS branch_id 
FROM branch 
WHERE branch.name = :name_1) AS b ON b.branch_id = e.assigned_branch_id"""
   account_id  cust_id   open_date product_cd
0           1        1  2000-01-15        CHK
1           2        1  2000-01-15        SAV
2           3        1  2004-06-30         CD
3           4        2  2001-03-12        CHK
4           5        2  2001-03-12        SAV
5          17        7  2004-01-12         CD
6          27       11  2004-03-22        BUS
[(1, 1, datetime.date(2000, 1, 15), 'CHK'), (2, 1, datetime.date(2000, 1, 15), 'SAV'), (3, 1, datetime.date(2004, 6, 30), 'CD'), (4, 2, datetime.date(2001, 3, 12), 'CHK'), (5, 2, 